In [1]:
# some useful mysklearn package import statements and reloads
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

# uncomment once you paste your mypytable.py into mysklearn package
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

# uncomment once you paste your myclassifiers.py into mysklearn package
import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

In [3]:
import pandas as pd
import numpy as np
from mysklearn.myclassifiers import MyDecisionTreeClassifier
from mysklearn.myevaluation import (
    stratified_kfold_split, 
    confusion_matrix, 
    accuracy_score,
    binary_precision_score, 
    binary_recall_score, 
    binary_f1_score
)



# Load and Explore Bitcoin Sentiment Dataset

In [5]:
# Load the bitcoin sentiment dataset
df = pd.read_csv('input_data/bitcoin_sentiment.csv')

# Print dataset shape
print("Dataset Shape:")
print(f"  Rows: {df.shape[0]}")
print(f"  Columns: {df.shape[1]}")
print()

# Print first few rows
print("First 5 Rows:")
print(df.head())
print()

# Print headers (column names)
print("Column Headers:")
print(df.columns.tolist())
print()

# Print label distribution (assuming last column is the label)
label_column = df.columns[-1]
print(f"Label Distribution ('{label_column}'):")
print(df[label_column].value_counts())
print()
print("Label Proportions:")
print(df[label_column].value_counts(normalize=True))

Dataset Shape:
  Rows: 1074
  Columns: 28

First 5 Rows:
   Unnamed: 0      timestamp      open      high       low     close  \
0           0  1669852800000  17165.44  17317.80  16855.00  16980.08   
1           1  1669939200000  16980.07  17108.25  16791.02  17094.71   
2           2  1670025600000  17094.25  17158.42  16863.58  16888.53   
3           3  1670112000000  16889.17  17199.99  16882.86  17108.90   
4           4  1670198400000  17108.90  17424.59  16865.22  16966.05   

         volume               datetime_utc  merge_date  domestic_series  ...  \
0  31798.991518  2022-12-01 00:00:00+00:00  2022-12-01            7.577  ...   
1  23096.436867  2022-12-02 00:00:00+00:00  2022-12-02            7.577  ...   
2  14081.450672  2022-12-03 00:00:00+00:00  2022-12-03            7.577  ...   
3  16961.108288  2022-12-04 00:00:00+00:00  2022-12-04            7.577  ...   
4  33618.451090  2022-12-05 00:00:00+00:00  2022-12-05            7.577  ...   

   total_non-marketable  trea

In [6]:
# Examine the weighted_sentiment column more closely
print("Weighted Sentiment Statistics:")
print(df['weighted_sentiment'].describe())
print()

# Check for missing values
print("Missing Values per Column:")
print(df.isnull().sum())
print()

# Data types
print("Data Types:")
print(df.dtypes)

Weighted Sentiment Statistics:
count    1074.000000
mean        0.347973
std         0.274657
min        -0.749771
25%         0.171151
50%         0.376796
75%         0.540075
max         0.952912
Name: weighted_sentiment, dtype: float64

Missing Values per Column:
Unnamed: 0                                        0
timestamp                                         0
open                                              0
high                                              0
low                                               0
close                                             0
volume                                            0
datetime_utc                                      0
merge_date                                        0
domestic_series                                   0
federal_financing_bank                            0
foreign_series                                    0
government_account_series                         0
government_account_series_inflation_securities    0
spec

In [7]:
# Create binary classification label from weighted_sentiment
# Positive sentiment (>0) vs Negative/Neutral sentiment (<=0)
df['sentiment_label'] = df['weighted_sentiment'].apply(lambda x: 'Positive' if x > 0 else 'Negative/Neutral')

print("Classification Label Distribution:")
print(df['sentiment_label'].value_counts())
print()
print("Label Proportions:")
print(df['sentiment_label'].value_counts(normalize=True))
print()
print(f"Total instances: {len(df)}")

Classification Label Distribution:
sentiment_label
Positive            949
Negative/Neutral    125
Name: count, dtype: int64

Label Proportions:
sentiment_label
Positive            0.883613
Negative/Neutral    0.116387
Name: proportion, dtype: float64

Total instances: 1074
